In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


in the above cell: imports

# **1.Loading data**
loading testing and training data

In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


replacing the string values with numerical values to be easier in processing

In [4]:
replace_with_nums = { "Embarked": {"S": 0, "C": 1, "Q": 2 },"Sex": {"male": 0, "female": 1}}
train_data.replace(replace_with_nums, inplace=True)
test_data.replace(replace_with_nums, inplace=True)

# **2.Pre-processing**

# 2.1 Training data pre-processing

getting information about training data to know what is needed in the pre-processing phase

In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    int64  
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    float64
dtypes: float64(3), int64(6), object(3)
memory usage: 83.7+ KB


In [6]:
train_data.describe()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,889.000000
mean,446.000000,0.383838,2.308642,0.352413,29.699118,0.523008,0.381594,32.204208,0.362205
std,257.353842,0.486592,0.836071,0.477990,14.526497,1.102743,0.806057,49.693429,0.636157
min,1.000000,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,0.000000,20.125000,0.000000,0.000000,7.910400,0.000000
50%,446.000000,0.000000,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,0.000000
75%,668.500000,1.000000,3.000000,1.000000,38.000000,1.000000,0.000000,31.000000,1.000000
max,891.000000,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,2.000000


# 2.1.1 Checking for null values

In [7]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

get percentage of null values in each column to know how to handle them

In [8]:
print("% of Cabin null values: ", (train_data['Cabin'].isnull().sum()/891)*100)
print("% of Age null values: ",(train_data['Age'].isnull().sum()/891)*100)
print("% of Age null values: ",(train_data['Embarked'].isnull().sum()/891)*100)

% of Cabin null values:  77.10437710437711
% of Age null values:  19.865319865319865
% of Age null values:  0.22446689113355783


columns with null values more than 30% should be omitted, so i will drop cabin column

drop cabin column:

In [9]:
train_data = train_data.drop(['Cabin'], axis=1)

Age has less nan values than 30%, so i chose to replace nan values with column mean

In [10]:
train_data['Age'].fillna((train_data['Age'].mean()), inplace=True)

Embarked has only 2 nan values so i will drop their rows as dropping them won't affect the results

In [11]:
train_data.dropna(subset=['Embarked'], inplace=True)

recheck that all our data are filled

In [12]:
train_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

# 2.1.2 drop unique features

In [13]:
train_data = train_data.drop(['PassengerId'], axis=1)

checking for duplicates in data after dropping the id column that made each row unique

# 2.1.3 split training data to training data and validation data

using shuffle=True prevents data from overfitting

In [14]:
from sklearn.model_selection import train_test_split
training_data, validation_data = train_test_split(
    train_data, test_size=0.3,  random_state=40,shuffle=True)

# 2.2 Testing data pre-processing

get info about testing data

In [15]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    int64  
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 36.0+ KB


In [16]:
test_data.describe()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,418.000000,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000,418.000000
mean,1100.500000,2.265550,0.363636,30.272590,0.447368,0.392344,35.627188,0.464115
std,120.810458,0.841838,0.481622,14.181209,0.896760,0.981429,55.907576,0.685516
min,892.000000,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,0.000000,21.000000,0.000000,0.000000,7.895800,0.000000
50%,1100.500000,3.000000,0.000000,27.000000,0.000000,0.000000,14.454200,0.000000
75%,1204.750000,3.000000,1.000000,39.000000,1.000000,0.000000,31.500000,1.000000
max,1309.000000,3.000000,1.000000,76.000000,8.000000,9.000000,512.329200,2.000000


check for null values in the testing

In [17]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

as training data, drop cabin column and fill age with mean value

In [18]:
test_data = test_data.drop(['Cabin'], axis=1)
test_data['Age'].fillna((test_data['Age'].mean()), inplace=True)

fill fare column using interpolate which fill it with the average of the value before and the value after it, but it can't be used if there's more than two consecutive nan 

In [19]:
test_data['Fare'].interpolate(method='linear', inplace=True)

# **3.Training**

importing the model that will be used

# 3.1 choosing the model

In [20]:
from sklearn.ensemble import RandomForestClassifier

# 3.2 Loading data

specify the parameters used

In [21]:
y = training_data["Survived"] # selecting y in training phase which is similar to the output feature we want to predict
y_testing= validation_data["Survived"] #the output feature we want to predict
features = ["Pclass", "Sex", "SibSp", "Parch", "Fare"] #important features that will be considered
#getting the data: 
X = pd.get_dummies(training_data[features]) 
X_testing= pd.get_dummies(validation_data[features])
X_test = pd.get_dummies(test_data[features])

# 3.3 Parameters tuning

this step is done similar to the previous assignment
2 for loops are used to get the best combination between these two parameters: n_estimators which is the number of trees that will be used and max_depth is how deep the tree will go

In [22]:
candidate_max_depth = [2,3,4,5,7]
candidate_n_estimators = [40,50,70,100,120,150]
# Write loop to find the ideal tree size from candidate_max_leaf_nodes
for max_depth in candidate_max_depth:
    for n_estimators in candidate_n_estimators:
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=1)
        model.fit(X,y)
        print("at depth %d, and n_estimators %d" %(max_depth, n_estimators))
        print(model.score(X_testing, y_testing))


at depth 2, and n_estimators 40
0.8052434456928839
at depth 2, and n_estimators 50
0.8052434456928839
at depth 2, and n_estimators 70
0.8089887640449438
at depth 2, and n_estimators 100
0.8052434456928839
at depth 2, and n_estimators 120
0.8052434456928839
at depth 2, and n_estimators 150
0.8052434456928839
at depth 3, and n_estimators 40
0.8202247191011236
at depth 3, and n_estimators 50
0.8164794007490637
at depth 3, and n_estimators 70
0.8164794007490637
at depth 3, and n_estimators 100
0.8164794007490637
at depth 3, and n_estimators 120
0.8127340823970037
at depth 3, and n_estimators 150
0.8202247191011236
at depth 4, and n_estimators 40
0.8164794007490637
at depth 4, and n_estimators 50
0.8164794007490637
at depth 4, and n_estimators 70
0.8127340823970037
at depth 4, and n_estimators 100
0.8164794007490637
at depth 4, and n_estimators 120
0.8164794007490637
at depth 4, and n_estimators 150
0.8164794007490637
at depth 5, and n_estimators 40
0.8127340823970037
at depth 5, and n_esti

there are three combinations that are equal but for me the best choice is 150, 3

fitting the model and printing the final score, using random state = 1 to prevent overfitting 

In [23]:
 model = RandomForestClassifier(n_estimators=150, max_depth=3, random_state=1)
 model.fit(X,y)
 print(model.score(X_testing, y_testing))

0.8202247191011236


predicting and saving data

In [24]:

predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
